In [2]:
# -*- coding: utf-8 -*-
"""GridSearchCV_PlantCLEF2013.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1aZBAmZMLizQ6dA9POQHLiQ-a7AHY0Fxc
"""

from sklearn import metrics
import pandas as pd
import numpy as np
import glob
from sklearn.decomposition import PCA
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import GridSearchCV
from sklearn.multiclass import OneVsRestClassifier
import time
import collections
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from collections import defaultdict

# Commented out IPython magic to ensure Python compatibility.
from google.colab import drive
drive.mount('/gdrive/')
%cd /gdrive/My\ Drive/artigo_plantas
!ls


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive/
/gdrive/My Drive/artigo_plantas
nasnetlarge_features.csv  plantclef2013_vgg19.csv   resnet50_features.csv
plantclef2013_vgg16.csv   resnet152v2_features.csv


In [0]:
def multi_clf_metrics(arq_csv):

	# Cria X (features) e y (labels)
  especies = pd.read_csv(arq_csv, low_memory=False)
  X = np.array(especies.iloc[2:,0:4096])
  y = np.array(especies.iloc[2:,4096])		
  # Executa o PCA
  pca = PCA(.90)
  principal_components = pca.fit_transform(X)
 # 'SVC - (linear, ovr)': OneVsRestClassifier(svm.SVC(kernel='linear', degree=3, coef0=0.0,
#											shrinking=True, probability=False, tol=0.001, cache_size=500,
#											class_weight=None, verbose=False, max_iter=-1,
#											decision_function_shape='ovr', random_state=None)),					'SVC - (linear, ovo)': svm.SVC(kernel='linear', degree=3, coef0=0.0,
#											shrinking=True, probability=False, tol=0.001, cache_size=500,
#											class_weight=None, verbose=False, max_iter=-1,
#											decision_function_shape='ovo', random_state=None),					'SVC - (polynomial, ovo)': svm.SVC(kernel='poly', degree=3, coef0=0.0,
#											shrinking=True, probability=False, tol=0.001, cache_size=500,
#											class_weight=None, verbose=False, max_iter=-1,
#											decision_function_shape='ovo', random_state=None),
#					'SVC - (sigmoid, ovr)': OneVsRestClassifier(svm.SVC(kernel='sigmoid', degree=3,coef0=0.0,
#											shrinking=True, probability=False, tol=0.001, cache_size=500,
#											class_weight=None, verbose=False, max_iter=-1,
#											decision_function_shape='ovr', random_state=None)),
#					'SVC - (sigmoid, ovo)': svm.SVC(kernel='sigmoid', degree=3, coef0=0.0,
#											shrinking=True, probability=False, tol=0.001, cache_size=500,
#											class_weight=None, verbose=False, max_iter=-1,
#											decision_function_shape='ovo', random_state=None)
  classifiers = {
					'SVC - (polynomial, ovr)': OneVsRestClassifier(svm.SVC(kernel='poly', degree=3, coef0=0.0,
											shrinking=True, probability=False, tol=0.001, cache_size=2000,
											class_weight=None, verbose=False, max_iter=-1,
											decision_function_shape='ovr', random_state=None))
}

  scoring = ['f1_micro']
  # Parametros para rodar com o grid search
  parameters = {"estimator__C" : [0.01, 0.1, 10], 'estimator__gamma':[0.0001, 0.001, 0.01]}
  results = []
  f = open('gd_srch_results_'+arq_csv,'w')
  f.write('classifier name,'+'best parameters,'+'all results\n')
  f.close()
  for name_clf, clf in classifiers.items():
    print('Executando classificador'+ name_clf)
    scores = GridSearchCV(clf, parameters,cv=5, scoring=scoring, refit='f1_micro', return_train_score=True, n_jobs=-1, verbose=51)
    result = scores.fit(principal_components,y)
    results.append(result)
    print('Melhor configuração: '+result.best_params_)
    f = open('gd_srch_results_'+arq_csv,'w')
    f.write(name_clf+',')
    f.write(result.best_params_+',')
    f.write(result+'\n')
    f.close()
  return result

In [0]:
results = multi_clf_metrics('plantclef2013_vgg16.csv')


Executando classificadorSVC - (polynomial, ovr)
Fitting 5 folds for each of 9 candidates, totalling 45 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
Memmapping (shape=(26077, 527), dtype=float64) to new file /dev/shm/joblib_memmapping_folder_121_4253752101/121-140674405969312-cc2a0eeebe14444aaedaee28aa2ea8c8.pkl
